In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import torch.nn as nn
import numpy as np
import random
from tqdm import tqdm
import scanpy as sc
import torch.nn.functional as F
from collections import defaultdict
from torch.distributions import Normal, kl_divergence
from torch.optim import lr_scheduler
import scanpy as sc
import numpy as np
import pandas as pd
# from umap.umap_ import UMAP
# from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cudf
from cuml import PCA, UMAP
import cupy as cp

from model_VAE import VAE
from dataloader_VAE import get_h5ad_data, get_dataloader, normalize, inverse_normalize
from train_VAE import train_vae

# 准备数据

In [4]:
# 准备数据

data_list = get_h5ad_data('drosophila_scNODE2_2000genes_2489cells_11tps.h5ad')
norm_data_list, scalers = normalize(data_list)


# 训练集和测试集对应的下标
all_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
train_index = [0, 1, 2, 3, 5, 7, 9, 10]
test_index = [4, 6, 8]

train_list = [norm_data_list[i] for i in train_index]
test_list = [norm_data_list[i] for i in test_index]

train_dataloader = get_dataloader(train_list, train_index, batch_size=64)
test_dataloader = get_dataloader(test_list, test_index, batch_size=64)
all_dataloader = get_dataloader(norm_data_list, all_index, batch_size=64)

# 模型训练

In [5]:
device = "cuda:0"
net = VAE().to(device)

num_epoch = 20
train_vae(net, all_dataloader, num_epoch, device=device)

# 保存模型
model_path = '/home/hanyuji/Results/VAE_result/vae_model_0603_DR_all.pt'
torch.save(net.state_dict(), model_path)

100%|█████████| 20/20 [01:18<00:00,  3.94s/it, recon_loss=109.551,kl_loss=3.291]


# 模型生成

In [6]:
# 加载模型
device = "cuda:0"
net = VAE().to(device)
model_path = '/home/hanyuji/Results/VAE_result/vae_model_0603_DR_all.pt'
net.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [8]:
# 准备存放输出
latent_list_mix=[]
latent_dict = {}  # 12个数组 [], [], [], [], [], [], [], [], [], [], [], []
recon_dict = {}  # 12个数组

# {4: [], 6: [], 8: []}
# for index in test_index:
for index in all_index:
    
    latent_dict[index] = []
    recon_dict[index] = []

In [9]:
net.eval()
# for (x, y) in test_dataloader:
for (x, y) in all_dataloader:
    
    x = x.float().to(device)
    z, mu, var = net.encoder(x)
    recon = net.decoder(z)
    
    z_np = z.detach().cpu().numpy()
    y_np = y.detach().cpu().numpy()
    recon_np = recon.detach().cpu().numpy()
        
    for (z_i,y_i) in zip(z_np,y_np):
        latent_dict[y_i].append(z_i)        
    for (recon_i,y_i) in zip(recon_np,y_np):
        recon_dict[y_i].append(recon_i)
       

In [10]:
latent_list = []
recon_list = []
label_list = []
for index, arr in latent_dict.items():
    latent_list.append(np.asarray(arr))
    for _ in arr:
        label_list.append(index)
    
for index, arr in recon_dict.items():
    recon_list.append(np.asarray(arr))


In [13]:
import pickle
# 保存这些数组到一个文件

file_name = 'DR_latent_50.pkl'
with open('/home/hanyuji/Results/VAE_result/'+file_name, 'wb') as f:
    pickle.dump(latent_list, f)

# # 从文件加载这些数组
# with open('array_list.pkl', 'rb') as f:
#     loaded_array_list = pickle.load(f)

# # 验证加载的数组与原始数组是否相同
# for original, loaded in zip(latent_list, loaded_array_list):
#     print(np.array_equal(original, loaded))